# Example: Modeling the Price of European Call or Put Contracts

## Background
A `call` option is a financial contract that gives the holder the right, but not the obligation, to sell a specified asset, such as stocks, commodities, or currencies, at a predetermined price within a specified time period. Let's consider stock as the underlying asset. A single standard `put` contract controls `100` shares of stock.

The business case for buying (or selling) a `call` contract:
* __Buyer (long)__: From the buyer's perspective, call contracts allow an investor to benefit from the price movement of `XYZ` to the upside _without_ purchasing `XYZ`. Further, call options (again from the buyer's perspective) have _limited downside risk_, i.e., the maximum amount that the holder of the call option can lose is the premium paid for the option. Finally, call options are a mechanism to purchase shares of `XYZ` at the strike price of $K$ instead of the market price of $S$. 
* __Seller (short)__: From the seller's perspective, the main objective of selling a call contract is to collect the premium $\mathcal{P}$. Call contracts also allow the seller to benefit from the price movement of `XYZ` to the downside _without_ purchasing `XYZ`. However, for a seller, call options have _unlimted upside_ risk; thus, call options are often only sold by investors who already own the required number of shares of `XYZ` (known as a [covered call position](https://www.investopedia.com/terms/c/coveredcall.asp)). Finally, call options offer the seller the opportunity to sell shares of `XYZ` at the strike price of $K$ instead of the market price of $S$.

A `put` option is a financial contract that gives the holder the right, but not the obligation, to sell a specified asset, such as stocks, commodities, or currencies, at a predetermined price within a specified time period. Let's consider stock as the underlying asset. A single standard `put` contract controls `100` shares of stock.

The business case for buying (or selling) `put` contracts:
* __Buyer (long)__: From the buyer's perspective, `put` contracts allow an investor to benefit from the price movement of `XYZ` to the downside _without_ purchasing `XYZ`. Further, `put` options (again from the buyer's perspective) have _limited downside risk_, i.e., the maximum amount that the holder of the `put` option can lose is the premium paid for the option. Finally, `put` contracts are a mechanism to sell shares of `XYZ` at the strike price of $K$ instead of the market price of $S$. 
* __Seller (short)__: From the seller's perspective, the motivation for selling a `put` contract is to collect the premium $\mathcal{P}$. Put contracts also allow the seller to benefit from the price movement of `XYZ` to the upside _without_ purchasing `XYZ`. However, for a seller, `put` options have _unlimted downside_ risk; thus, `put` options are often only sold by investors who have set aside the required capital to purchase the required number of shares of `XYZ` (known as a [cash secured put position](https://www.fidelity.com/learning-center/investment-products/options/know-about-cash-covered-puts)). Finally, `put` options offer the seller the opportunity to buy shares of `XYZ` at the strike price of $K-\mathcal{P}$ instead of the market price of $S$.


Buyers of [European](https://www.investopedia.com/terms/e/europeanoption.asp) style contracts can only exercise their right on the expiration date. 

## Setup
Fill me in

In [1]:
include("Include.jl");

## Learning objectives
In this example, we will calculate the premium, profit and breakeven for `call` and `put` option contracts. Our focus will be on [European-style contracts](https://en.wikipedia.org/wiki/Option_style), which can only be exercised at the time of expiration.

* __Objective 1__: involves computing the premium, profit, and breakeven points for European-style `call` contracts using both the [Monte Carlo simulation approach](https://en.wikipedia.org/wiki/Monte_Carlo_method) and the [Black-Scholes pricing formula](https://en.wikipedia.org/wiki/Black–Scholes_mode). 
* __Objective 2__: involves computing the premium, profit, and breakeven points for European-style `put` contracts using both the [Monte Carlo simulation approach](https://en.wikipedia.org/wiki/Monte_Carlo_method) and the [Black-Scholes pricing formula](https://en.wikipedia.org/wiki/Black–Scholes_mode). 

## Objective 1: European `call` contracts

Let's begin by computing the premium $\mathcal{P}_{c}(K,S(T))$ the buyer must pay for a `call` contract. When early excersise is not allowed, the pricing formula is given by:

$$\mathcal{P}_{c}(K,S(T)) = \mathbb{E}\Bigl(\mathcal{D}^{-1}_{T,0}(\bar{r})\cdot{V_{c}}(K,S(T))\Bigr)$$

which says the right (but not the obligation) to excercise the `call` contract is the expected value of the discounted future payoff from the contract. To compute the expectation, we simulate the future share price `T` days in the future using a [geometric Brownian motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion#:~:text=A%20geometric%20Brownian%20motion%20(GBM,a%20Wiener%20process)%20with%20drift.) model for the future share price:

$$S(T) = S_{\circ}\exp\Biggl[\left(\bar{r}-\frac{\sigma^{2}}{2}\right)\cdot{T} + (\sigma\sqrt{T})\cdot{Z_{t}(0,1)}\Biggr]$$

where we assume $S_{\circ}$ is the share price today, $\bar{r}$ denotes the risk-free rate (risk-neutral pricing), $\sigma$ denotes share price volatility and `T` denotes the number of days until contract expiration. Once we have the $\mathcal{P}_{c}(K,S(T))$ value, we can calculate (and visualize) the payoff, potential profit and breakeven values for the `call` contract.

### Implementation
Let's consider an example: We have a European `call` that expires in `T = 365` days with a strike price `K = 60.0`. The current share price is `S(0)=60.0`. Assume a risk free rate of return of `5%` and a implied volatility of `10%`. The correct answer is $\mathcal{P}_{c}(K,S(T))$ = 4.08 USD/share. 

Let's compute the European `call` premium using two appoaches:

* Approach 1: Monte Carlo simulation of the share price by sampling the geometric Brownian motion model, and then explicitly calculating the expectation in the pricing relationship
* Approach 2: Use the [Black–Scholes](https://en.wikipedia.org/wiki/Black–Scholes_model) pricing formula for a European `call` option

We start by defining the problem parameters that are shared between the approaches:

In [2]:
Δt = (1.0/365.0);
Sₒ = 60.0;
K = 60.0;
T = 365.0*Δt;
r̄ = 0.05;
σ̄ = 0.10;

#### Approach 1: Monte Carlo estimate of European `call` contract price
Let's build an instance of the `MyGeometricBrownianMotionEquityModel` type which holds the value for the price simulation using the `build(...)` method and store in the `model` variable:

In [3]:
model = build(MyGeometricBrownianMotionEquityModel, (
        μ = r̄, σ = σ̄));

Next, we build an instance of the `MyEuropeanCallContractModel` type which holds the parameters for the `call` contract using the `build(...)` method. We store the contract model in the `call_contract_model` variable:

In [4]:
call_contract_model = build(MyEuropeanCallContractModel, (
        K = K, IV = σ̄, DTE = T, sense = 1));

We'll simulate the future share price at expiration $S(T)$ for different number of sample paths. We'll store these values in the `number_of_samples` array:

In [5]:
number_of_samples = range(1.0,stop=5,step=1.0) |> collect |> (x-> exp10.(x)) |> (x-> 5*Int.(x));

Finally, for each number of sample paths, we sample the geometric Brownian model instance using the `sample(...)` function, compute the payoff using `payoff(...)` function, compute the premium array which is stored in the `P̄` variable, and then compute the mean `premium`  and standard error `SE` of the expected discounted future contract payoff. We propulate a `DataFrame` instance that holds the data for each number of sample paths:

In [6]:
call_price_df = DataFrame(n = Int64[], premium = Float64[], SE = Float64[], CI95 = Float64[]);
for n ∈ number_of_samples
    S = sample_endpoint(model, (T = T, Sₒ = Sₒ), number_of_paths = n);
    P = payoff([call_contract_model], S);
    P̄ = (1/𝒟(r̄,T))*P[:,3];
    mean_value = mean(P̄);
    std_error_value = (1.0/sqrt(n))*std(P̄);
    CI95_value = 1.96*std_error_value;
    
    results_df = (
        
        n = n,
        premium = mean_value,
        SE = std_error_value,
        CI95 = CI95_value
    );
    
    push!(call_price_df, results_df);
end

call_price_df

Row,n,premium,SE,CI95
,Int64,Float64,Float64,Float64
1,50,3.51091,0.561407,1.10036
2,500,4.35202,0.215787,0.422943
3,5000,4.21355,0.0662643,0.129878
4,50000,4.07984,0.0207828,0.0407343
5,500000,4.07962,0.00655661,0.012851


#### Approach 2: Black–Scholes pricing formula for a European `call` contract
The [Black–Scholes pricing formula ](https://en.wikipedia.org/wiki/Black–Scholes_model) for a European `call` option is given by the expression:

$$\mathcal{P}_{c}(K,S(T)) = N(d_{+})S_{\circ} - N(d_{-})K\mathcal{D}^{-1}_{T,0}(\bar{r}) - $$

where:

$$
\begin{eqnarray}
d_{+} & = & \frac{1}{\sigma\sqrt{T}}\left[\ln(\frac{S_{\circ}}{K}) + (r+\frac{\sigma^{2}}{2})T\right] \\
d_{-} & = & d_{+} - \sigma\sqrt{T}
\end{eqnarray}
$$

and $N(\dots)$ denotes the standard normal cumulative distribution function.

In [7]:
d₊ = (1/σ̄*sqrt(T))*(log(Sₒ/K)+(r̄+(σ̄^2)/2)*T);
d₋ = d₊ - σ̄*sqrt(T);
P = (cdf(Normal(0,1), d₊)*Sₒ - cdf(Normal(0,1), d₋)*K*(1/𝒟(r̄,T))) |> x-> round(x,sigdigits=4)
println("The premium for the European call contract computed by Black-Scholes is: $(P) USD/share")

The premium for the European call contract computed by Black-Scholes is: 4.083 USD/share


## Objective 2: European `put` contracts

Let's begin by computing the premium $\mathcal{P}_{p}(K,S(T))$ the buyer must pay for a `put` contract. When early excersise is not allowed, the pricing formula is given by (the equality case):

$$\mathcal{P}_{p}(K,S(T)) = \mathbb{E}\Bigl(\mathcal{D}^{-1}_{T,0}(\bar{r})\cdot{V_{p}}(K,S(T))\Bigr)$$

which says the right (but not the obligation) to excercise the `put` contract is the expected value of the discounted future payoff from the contract. To compute the expectation, we simulate the future share price `T` days in the future using a [geometric Brownian motion](https://en.wikipedia.org/wiki/Geometric_Brownian_motion#:~:text=A%20geometric%20Brownian%20motion%20(GBM,a%20Wiener%20process)%20with%20drift.) model for the future share price:

$$S(T) = S_{\circ}\exp\Biggl[\left(\bar{r}-\frac{\sigma^{2}}{2}\right)\cdot{T} + (\sigma\sqrt{T})\cdot{Z_{t}(0,1)}\Biggr]$$

where we assume $S_{\circ}$ is the share price today, $\bar{r}$ denotes the risk-free rate (risk-neutral pricing), $\sigma$ denotes share price volatility and `T` denotes the number of days until contract expiration. Once we have the $\mathcal{P}_{p}(K,S(T))$, we can calculate (and visualize) the payoff and potential profit for the `put` contract.

### Implementation
Let's consider an example: We have a European `put` that expires in `T = 365` days with a strike price `K = 60.0`. The current share price is `S(0)=60.0`. Assume a risk free rate of return of `5%` and a implied volatility of `10%`. The correct answer is $\mathcal{P}_{p}(K,S(T))$ = 1.16 USD/share. Let's compute the premium using two appoaches:

* Approach 1: Monte Carlo simulation of the share price by sampling the geometric Brownian motion model, and then explicitly calculating the expectation in the pricing relationship
* Approach 2: Use the [Black–Scholes](https://en.wikipedia.org/wiki/Black–Scholes_model) pricing formula for a European `put` option

We start by defining the problem parameters that are shared between the approaches: